In [1]:
import numpy as np
from astropy.table import Table
import Euclid
from tqdm.notebook import tqdm

%elogin
%erun Elements 5.10

sys.path.append('/home/aalvarez/Work/Projects/PHZ_NNPZ/PHZ_NNPZ/nnpz/python/')
from nnpz.reference_sample.ReferenceSample import ReferenceSample

In [2]:
ref = Table.read('/home/aalvarez/Phosphoros/Results/WillPP/Reference/phz_cat.fits')
ref = ref[ref['ID'].argsort()]

In [3]:
gal_types = np.array(['Elliptical', 'Spiral', 'Starforming'])
sfh_types = np.array(['Exponentially declining', 'Delayed exponential'])

sed_class = dict(
    Ell1_A_0 = (0, 0, 0.3),
    Ell2_A_0 = (0, 0, 0.3),
    Ell3_A_0 = (0, 0, 0.7),
    Ell4_A_0 = (0, 0, 0.7),
    Ell5_A_0 = (0, 0, 1.5),
    Ell6_A_0 = (0, 0, 1.5),
    Ell7_A_0 = (0, 0, 2.5),
    S0_A_0   = (1, 0, 2.5),
    Sa_A_0   = (1, 0, 5.0),
    Sa_A_1   = (1, 0, 5.0),
    Sb_A_0   = (1, 0, 5.0),
    Sb_A_1   = (1, 0, 5.0),
    Sc_A_0   = (1, 0, 10.0),
    Sc_A_1   = (1, 0, 10.0),
    Sc_A_2   = (1, 0, 10.0),
    Sd_A_0   = (1, 1, 0.3),
    Sd_A_1   = (1, 1, 0.3),
    Sd_A_2   = (1, 1, 0.3),
    Sdm_A_0  = (1, 1, 0.3),
    SB0_A_0  = (2, 1, 1.5),
    SB10_A_0 = (2, 1, 10.0),
    SB11_A_0 = (2, 1, 10.0),
    SB1_A_0  = (2, 1, 2.5),
    SB2_A_0  = (2, 1, 2.5),
    SB3_A_0  = (2, 1, 2.5),
    SB4_A_0  = (2, 1, 2.5),
    SB5_A_0  = (2, 1, 5.0),
    SB6_A_0  = (2, 1, 5.0),
    SB7_A_0  = (2, 1, 5.0),
    SB8_A_0  = (2, 1, 5.0),
    SB9_A_0  = (2, 1, 10.0),
)

In [4]:
ref_gal_type = []
ref_sfh_type = []
ref_sfh_tau = []
for sed in ref['SED']:
    sed = sed.split('/')[1].strip()
    gal_type, sfh_type, sfh_tau = sed_class[sed]
    ref_gal_type.append(gal_type)
    ref_sfh_type.append(sfh_type)
    ref_sfh_tau.append(sfh_tau)
ref_gal_type = np.array(ref_gal_type)
ref_sfh_type = np.array(ref_sfh_type)
ref_sfh_tau = np.array(ref_sfh_tau)

In [5]:
idx = np.load('/home/aalvarez/Work/Data/WillPP/ReferenceSample/index.npy', mmap_mode='r')

In [6]:
(idx['id'] == ref['ID']).all()

True

In [7]:
npps = []
for i in [1,2]:
    pp = np.load(f'/home/aalvarez/Work/Data/WillPP/ReferenceSample/pp_data_{i}.npy', mmap_mode='r')
    dtypes = dict([(n, pp.dtype[n]) for n in pp.dtype.names])
    dtypes['GAL_TYPE'] = np.int64
    dtypes['SFH_TYPE'] = np.int64
    npp = np.zeros(pp.shape, dtype=list(dtypes.items()))
    for c in pp.dtype.names:
        npp[c] = pp[c]
    npp['GAL_TYPE'] = ref_gal_type[idx['pp_file'] == i, np.newaxis]
    npp['SFH_TYPE'] = ref_sfh_type[idx['pp_file'] == i, np.newaxis]
    npp['SFH_TAU']  = ref_sfh_tau[idx['pp_file'] == i, np.newaxis]
    npps.append(npp)

In [8]:
for i in [1,2]:
    np.save(f'/home/aalvarez/Work/Data/WillPP/ReferenceSample/pp_data_{i}.npy', npps[i-1])

In [9]:
sample = ReferenceSample('/home/aalvarez/Work/Data/WillPP/ReferenceSample/', providers={'MontecarloProvider': {'name': 'pp', 'data': 'pp_data_{}.npy'}})

INFO    : Found provider MontecarloProvider
2020-11-16T14:04:31CET ReferenceSample  INFO : Found provider MontecarloProvider


In [10]:
prov = sample.getProvider('pp')
for j, i in tqdm(list(enumerate(ref['ID']))):
    assert (prov.getData(i)['GAL_TYPE'] == ref_gal_type[j]).all()
    assert (prov.getData(i)['SFH_TYPE'] == ref_sfh_type[j]).all()
    assert (prov.getData(i)['SFH_TAU'] == ref_sfh_tau[j]).all()